**Code 3 -
Create a Complete Machine by Merged All Part Together -
Mohamadreza Jafaei -
400131017 ***

In [1]:
!pip install transformers
!pip install annoy
!pip install datasets transformers
!pip install hazm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!wget https://github.com/sobhe/hazm/releases/download/v0.5/resources-0.5.zip
!wget https://github.com/sobhe/hazm/releases/download/v0.5/resources-stanford.zip
!unzip resources-0.5.zip -d resources
!unzip resources-stanford.zip -d resources

In [3]:
import pandas as pd
import torch
import annoy
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoConfig, AutoModel, TFAutoModel
from tqdm import tqdm
from datasets import load_dataset, load_metric
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from transformers import pipeline
from hazm import *
import copy

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
documents_df = pd.read_pickle("/content/drive/MyDrive/CNN/documents_embeding_df_full.pkl")
contexts_df = pd.read_pickle("/content/drive/MyDrive/CNN/context_query_question_embeding_df_full.pkl")

In [6]:
contexts_df.columns

Index(['level_0', 'index', 'id', 'title', 'context', 'question', 'answers',
       'search_query', 'search_query_embed', 'question_embed',
       'context_embed'],
      dtype='object')

In [7]:
contexts_df = contexts_df.drop(['search_query_embed'], axis=1)
contexts_df = contexts_df.drop(['question_embed'], axis=1)

In [36]:
create_query_from_question = True
number_of_documets_retrival = 3
number_of_contexts_retrival = 3

In [61]:
model_names = [
    "xlm-roberta-base", # 0
    "HooshvareLab/bert-fa-zwnj-base", # 1
    "HooshvareLab/distilbert-fa-zwnj-base", # 2
    "HooshvareLab/roberta-fa-zwnj-base", # 3
]
model_id = 1
model_name_or_path = model_names[model_id]

config = AutoConfig.from_pretrained(model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, do_lower_case=True)
model = AutoModel.from_pretrained(model_name_or_path)
model.eval()
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

model.to(device)


# mrr_model = pipeline(task = "question-answering" ,model="/content/drive/MyDrive/CN/distilbert-fa-zwnj-base-on-pquad-dataset")
mrr_model = pipeline(task = "question-answering" ,model="/content/drive/MyDrive/CN/xlm-roberta-base-on-pquad-dataset")

Some weights of the model checkpoint at HooshvareLab/bert-fa-zwnj-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at HooshvareLab/bert-fa-zwnj-base and are newly initialized: ['bert.pooler.dense.bias', 'bert.poo

In [38]:
def create_search_query(text):
    normalizer = Normalizer()
    lemmatizer = Lemmatizer()
    tagger = POSTagger(model='resources/postagger.model')
    chunker = Chunker(model='resources/chunker.model')
    # normalize = normalizer.normalize(text)
    words = word_tokenize(text)
    pos_tags = tagger.tag(words)
    tree = chunker.parse(pos_tags)
    query_words = []
    for subtree in tree.subtrees(filter=lambda t: t.label() == 'NP'):
        for word, pos in subtree.leaves():
            query_words.append(word)#(lemmatizer.lemmatize(word))
    query = ' '.join(query_words)
    return query

In [39]:
def embed_question(question):
  embeds_text = []
  question_tensor = tokenizer.encode(question, return_tensors='pt' ,add_special_tokens=True,truncation=True)
  question_tensor = question_tensor.to(device)
  with torch.no_grad():
      output = model(question_tensor)[0]
      mean_pooled = torch.mean(output, 1)
      question_embedding = mean_pooled.detach().cpu().numpy().tolist()[0]
  return question_embedding

In [40]:
question_id =121 #19999

In [41]:
Question = contexts_df["question"][question_id]
Answer = contexts_df["answers"][question_id]
title = contexts_df["title"][question_id]


In [42]:
print("Question = " + Question)
print("title = "+title)

Question = فوتبال به سبک امروزی در چه سالی آغاز شد؟
title = فوتبال


In [43]:
Answer

{'text': ['اوایل سال ۱۸۰۰ میلادی'], 'answer_start': [199]}

In [44]:
Query = create_search_query(Question)

Question_embed = embed_question(Question)
Query_embed = embed_question(Query)

In [45]:
Query

'فوتبال سبک امروزی چه سالی'

In [46]:
calculator = annoy.AnnoyIndex(768, metric='angular')
for index, embedding in zip(documents_df.index, documents_df['document_embeds']):
    calculator.add_item(index, embedding)
calculator.build(number_of_documets_retrival)

True

In [47]:
# Function to retrieve the most similar documents to a query
def retrieve_documents(documents_df,query,calculator,numbers):
    document_indices = calculator.get_nns_by_vector(query, numbers)
    titles = []
    for item in document_indices:
      titles.append(documents_df.iloc[item]["title"])
    return titles

In [48]:
titles = retrieve_documents(documents_df,Query_embed,calculator,number_of_documets_retrival)

In [49]:
titles

['سویا', 'فوتبال', 'راگبی فوتبال']

In [50]:
def get_contexts_of_titles(titles,contexts_df):
  paragraphs =pd.DataFrame()
  for title in titles:
    df = contexts_df[contexts_df["title"] == title]
    if paragraphs.empty:
      paragraphs = copy.deepcopy(df)
    else:
      paragraphs = paragraphs.append(df)
  paragraphs = paragraphs.reset_index(drop=True)
  return paragraphs

In [51]:
selected_paragraphs = get_contexts_of_titles(titles,contexts_df)

In [52]:
selected_paragraphs

,level_0,index,id,title,context,question,answers,search_query,context_embed
0,22056,22056,619713.0,سویا,باشگاه فوتبال سویا (به اسپانیایی: Sevilla Fút...,اولین و قدیمی‌ترین باشگاه فوتبال کشور اسپانیا ...,"{'text': ['سویا'], 'answer_start': [14]}",اولین و قدیمی‌ترین باشگاه فوتبال کشور اسپانیا ...,"[-1.1369779109954834, -0.6225090026855469, -0...."
1,22057,22057,619714.0,سویا,باشگاه فوتبال سویا (به اسپانیایی: Sevilla Fút...,بزرگ‌ترین شهر ایالت خودمختار آندلس در کجا واق...,"{'text': ['سویا'], 'answer_start': [137]}",بزرگ‌ترین شهر ایالت خودمختار آندلس کجا,"[-1.1369779109954834, -0.6225090026855469, -0...."
2,22058,22058,619715.0,سویا,باشگاه فوتبال سویا (به اسپانیایی: Sevilla Fút...,باشگاه فوتبال سویا در چه تاریخی توسط فردی اسکا...,"{'text': ['۲۵ ژانویه سال ۱۸۹۰ میلادی'], 'answe...",باشگاه فوتبال سویا چه تاریخی فردی اسکاتلندی نا...,"[-1.1369779109954834, -0.6225090026855469, -0...."
3,22059,22059,619716.0,سویا,باشگاه فوتبال سویا (به اسپانیایی: Sevilla Fút...,باشگاه فوتبال سویا توسط چه کسی تأسیس گردید؟,{'text': ['فردی اسکاتلندی به نام ادوارد فراکُه...,باشگاه فوتبال سویا چه کسی,"[-1.1369779109954834, -0.6225090026855469, -0...."
4,22060,22060,619717.0,سویا,باشگاه فوتبال سویا (به اسپانیایی: Sevilla Fút...,نخستین رئیس باشگاه سویا چه کسی بود؟,{'text': ['فردی اسکاتلندی به نام ادوارد فراکُه...,نخستین رئیس باشگاه سویا چه کسی,"[-1.1369779109954834, -0.6225090026855469, -0...."
...,...,...,...,...,...,...,...,...,...
287,79615,7619,1976541.0,راگبی فوتبال,در سال ۱۹۰۶ و با کاهش تعداد بازیکان هر تیم از ...,چه تعداد باشگاه در لیگ راگبی نیو ساوت ولز هستند؟,"{'text': [], 'answer_start': []}",چه تعداد باشگاه لیگ راگبی نیو ساوت ولز,"[-0.7444747686386108, -0.18539494276046753, -1..."
288,79616,7620,1976592.0,راگبی فوتبال,متأسفانه از آنجائیکه در ایران این ورزش را ب...,به چه دلیل از فوتبال آمریکایی در ایران حمایت ...,{'text': ['به دلیل مشکل سیاسی ایران باآمریکا'...,چه دلیل فوتبال آمریکایی ایران,"[-0.1241147443652153, 0.2722756266593933, 0.01..."
289,79617,7621,1976593.0,راگبی فوتبال,متأسفانه از آنجائیکه در ایران این ورزش را ب...,با پیگیری‌های چه کسی باعث شد تا ذهن مسئولین وق...,"{'text': ['علیرضا فضل‌الله اعرابی'], 'answer_s...",پیگیری‌های چه کسی ذهن مسئولین وقت ورزش ایران م...,"[-0.1241147443652153, 0.2722756266593933, 0.01..."
290,79618,7622,1976594.0,راگبی فوتبال,متأسفانه از آنجائیکه در ایران این ورزش را ب...,سلسه مقالات کدام روزنامه نگار و فعال حقوق بشر ...,"{'text': ['محمدصفر لفوتی'], 'answer_start': [2...",سلسه مقالات کدام روزنامه نگار و فعال حقوق بشر ...,"[-0.1241147443652153, 0.2722756266593933, 0.01..."


In [53]:
selected_paragraphs.to_csv('question1999_paragraphs.csv')

In [54]:
def unique(list1):
    unique_list = []
    for x in list1:
        if x not in unique_list:
            unique_list.append(x)
    return unique_list

In [55]:
# Function to retrieve the most similar documents to a query
def retrieve_contexts(contexts_df,unique_embeds,query,calculator,numbers):
    document_indices = calculator.get_nns_by_vector(query, numbers)
    paragraphs = []
    for item in document_indices:
      selected_embed = unique_embeds[item]
      for i in range(len(contexts_df)):
        if contexts_df.iloc[i]["context_embed"]==selected_embed:
          paragraphs.append(contexts_df.iloc[i]["context"])
          break
    return paragraphs

In [56]:
searcher = annoy.AnnoyIndex(768, metric='angular')
unique_embeds = unique(selected_paragraphs['context_embed'])
for index, embedding in enumerate(unique_embeds):
    searcher.add_item(index, embedding)
searcher.build(number_of_contexts_retrival)

True

In [57]:
paragraphs = retrieve_contexts(selected_paragraphs,unique_embeds,Question_embed,searcher,number_of_contexts_retrival)

In [58]:
paragraphs[0]

'پیشینهٔ معاصر این بازی جهانی به بیش از ۱۰۰ سال پیش بازمی\u200cگردد؛ زمانی که این ورزش در انگلستان شکل\u200cگرفت و اتحادیه فوتبال انگلستان به وجود آمد و فوتبال به ورزشی رسمی تبدیل\u200cشد. فوتبال به سبک امروزی، در اوایل سال ۱۸۰۰ میلادی در هفت مدرسهٔ عمومی انگلستان آغاز گردید. شش مدرسه تقریباً همان فوتبال به سبک قدیمی را اجرا می\u200cکردند؛ اما مدرسهٔ راگبی انگلستان که در سال ۱۵۶۷ تأسیس\u200cشده بود، نسخه\u200cای متفاوت از فوتبال را ارائه\u200cداد. مدارس دیگر نیز قوانینی را ساخته و در پایان، بازی فوتبال مدرن را بنیان\u200cگذاری کردند. در این سال اندازه و وزن توپ\u200cها نیز استاندارد شد. در نخستین بازی ۹۰ دقیقه\u200cای فوتبال، دو تیم لندن و شفیلد در سال ۱۸۶۶ به رقابت با یک\u200cدیگر پرداختند.'

In [62]:
outputs = []
for paragraph in paragraphs:
  outputs.append(mrr_model(question=Question, context=paragraph))

In [63]:
outputs

[{'score': 0.49756279587745667,
  'start': 199,
  'end': 220,
  'answer': 'اوایل سال ۱۸۰۰ میلادی'},
 {'score': 2.5414148563385197e-09,
  'start': 181,
  'end': 236,
  'answer': 'فوتبال آمریکایی، فوتبال استرالیایی، و فوتبال کانادایی.'},
 {'score': 4.5433682771545136e-07,
  'start': 115,
  'end': 126,
  'answer': '۱۸۰۰ میلادی'}]

In [33]:
outputs = []

context = "اسم من محمدرضا جفائی است. من بیست و چهار ساله هستم و دانشجوی دانشگاه امیرکبیر تهران هستم"
print(context)
print(mrr_model(question="اسم من چیست؟", context=context))
print(mrr_model(question="من چند سالمه؟", context=context))

اسم من محمدرضا جفائی است. من بیست و چهار ساله هستم و دانشجوی دانشگاه امیرکبیر تهران هستم
{'score': 0.9059651494026184, 'start': 7, 'end': 20, 'answer': 'محمدرضا جفائی'}
{'score': 0.028071196749806404, 'start': 29, 'end': 45, 'answer': 'بیست و چهار ساله'}
